## 175. Combine Two Tables (Easy)

Table: `Person`
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| PersonId    | int     |
| FirstName   | varchar |
| LastName    | varchar |
+-------------+---------+
```
PersonId is the primary key column for this table.
Table: `Address`
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| AddressId   | int     |
| PersonId    | int     |
| City        | varchar |
| State       | varchar |
+-------------+---------+
```
`AddressId` is the primary key column for this table.


Write a SQL query for a report that provides the following information for each person in the `Person` table, regardless if there is an address for each of those people:

`FirstName`, `LastName`, `City`, `State`

```mysql
SELECT p.FirstName, p.LastName, a.City, a.State
FROM Person p
LEFT JOIN Address a ON a.PersonId = p.PersonId;
```

## 176. Second Highest Salary (Easy)
Write a SQL query to get the second highest salary from the `Employee` table.
```
+----+--------+
| Id | Salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+
```
For example, given the above `Employee` table, the query should return 200 as the second highest salary. If there is no second highest salary, then the query should return null.
```
+---------------------+
| SecondHighestSalary |
+---------------------+
| 200                 |
+---------------------+
```

```mysql
select
    (select distinct salary
    from employee
    order by salary desc
    limit 1, 1) as SecondHighestSalary;
```


## 177. Nth Highest Salary (Medium)
Write a SQL query to get the nth highest salary from the `Employee` table.
```
+----+--------+
| Id | Salary |
+----+--------+
| 1  | 100    |
| 2  | 200    |
| 3  | 300    |
+----+--------+
```
For example, given the above `Employee` table, the nth highest salary where n = 2 is 200. If there is no nth highest salary, then the query should return null.
```
+------------------------+
| getNthHighestSalary(2) |
+------------------------+
| 200                    |
+------------------------+
```




```mysql
create function getNthHighestSalary(N int) returns int
begin
declare m int;
set m=n-1;
return (
    (select distinct salary
    from Employee
    order by salary desc
    limit m, 1)
);
end
```

## 185. Department Top Three Salaries (Hard)
The `Employee` table holds all employees. Every employee has an Id, and there is also a column for the department Id.
```
+----+-------+--------+--------------+
| Id | Name  | Salary | DepartmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 85000  | 1            |
| 2  | Henry | 80000  | 2            |
| 3  | Sam   | 60000  | 2            |
| 4  | Max   | 90000  | 1            |
| 5  | Janet | 69000  | 1            |
| 6  | Randy | 85000  | 1            |
| 7  | Will  | 70000  | 1            |
+----+-------+--------+--------------+
```
The `Department` table holds all departments of the company.
```
+----+----------+
| Id | Name     |
+----+----------+
| 1  | IT       |
| 2  | Sales    |
+----+----------+
```
Write a SQL query to find employees who earn the top three salaries in each of the department. For the above tables, your SQL query should return the following rows (order of rows does not matter).
```
+------------+----------+--------+
| Department | Employee | Salary |
+------------+----------+--------+
| IT         | Max      | 90000  |
| IT         | Randy    | 85000  |
| IT         | Joe      | 85000  |
| IT         | Will     | 70000  |
| Sales      | Henry    | 80000  |
| Sales      | Sam      | 60000  |
+------------+----------+--------+
```
Explanation:

In IT department, Max earns the highest salary, both Randy and Joe earn the second highest salary, and Will earns the third highest salary. There are only two employees in the Sales department, Henry earns the highest salary while Sam earns the second highest salary.



```mysql
select d.Name 'Department', e1.Name 'Employee', e1.Salary
from Employee e1
    join
    Department d on e1.DepartmentId = d.Id
where
    3 > (select count(distinct e2.salary)
    from Employee e2
    where e2.salary > e1.salary
        and e1.DepartmentId = e2.DepartmentId);
```

with `dense_rank()`
```mysql
select d.name department, tmp.name Employee, tmp.salary Salary
from (
    select departmentid, name, salary, dense_rank() over (partition by DepartmentId order by salary desc) ranks
    from employee) tmp
join department d
on tmp.departmentid = d.id
where ranks < 4;
```

## 178. Rank Scores (Medium)
Write a SQL query to rank scores. If there is a tie between two scores, both should have the same ranking. Note that after a tie, the next ranking number should be the next consecutive integer value. In other words, there should be no "holes" between ranks.
```
+----+-------+
| Id | Score |
+----+-------+
| 1  | 3.50  |
| 2  | 3.65  |
| 3  | 4.00  |
| 4  | 3.85  |
| 5  | 4.00  |
| 6  | 3.65  |
+----+-------+
```
For example, given the above `Scores` table, your query should generate the following report (order by highest score):
```
+-------+---------+
| score | Rank    |
+-------+---------+
| 4.00  | 1       |
| 4.00  | 1       |
| 3.85  | 2       |
| 3.65  | 3       |
| 3.65  | 3       |
| 3.50  | 4       |
+-------+---------+
```
Important Note: For MySQL solutions, to escape reserved words used as column names, you can use an apostrophe before and after the keyword. For example `Rank`.

Solution1:
```mysql
select score,
    dense_rank() over (order by score desc) "Rank"
from scores
```

Soulution2:
```mysql
select score,
    (select count(*)
    from (select distinct score s
        from scores) tmp where s >= score) "Rank"
from scores
order by score desc;
```

## 181. Employees Earning More than Their Manager (Easy)
The `Employee` table holds all employees including their managers. Every employee has an Id, and there is also a column for the manager Id.
```
+----+-------+--------+-----------+
| Id | Name  | Salary | ManagerId |
+----+-------+--------+-----------+
| 1  | Joe   | 70000  | 3         |
| 2  | Henry | 80000  | 4         |
| 3  | Sam   | 60000  | NULL      |
| 4  | Max   | 90000  | NULL      |
+----+-------+--------+-----------+
```
Given the Employee table, write a SQL query that finds out employees who earn more than their managers. For the above table, Joe is the only employee who earns more than his manager.
```
+----------+
| Employee |
+----------+
| Joe      |
+----------+
```

```mysql
select a.name "Employee"
from Employee a
join Employee b on a.ManagerId = b.Id
where a.salary > b.salary;
```


## 262. Trips and Users (Hard)
The `Trips` table holds all taxi trips. Each trip has a unique Id, while Client_Id and Driver_Id are both foreign keys to the Users_Id at the Users table. Status is an ENUM type of (‘completed’, ‘cancelled_by_driver’, ‘cancelled_by_client’).
```
+----+-----------+-----------+---------+--------------------+----------+
| Id | Client_Id | Driver_Id | City_Id |        Status      |Request_at|
+----+-----------+-----------+---------+--------------------+----------+
| 1  |     1     |    10     |    1    |     completed      |2013-10-01|
| 2  |     2     |    11     |    1    | cancelled_by_driver|2013-10-01|
| 3  |     3     |    12     |    6    |     completed      |2013-10-01|
| 4  |     4     |    13     |    6    | cancelled_by_client|2013-10-01|
| 5  |     1     |    10     |    1    |     completed      |2013-10-02|
| 6  |     2     |    11     |    6    |     completed      |2013-10-02|
| 7  |     3     |    12     |    6    |     completed      |2013-10-02|
| 8  |     2     |    12     |    12   |     completed      |2013-10-03|
| 9  |     3     |    10     |    12   |     completed      |2013-10-03|
| 10 |     4     |    13     |    12   | cancelled_by_driver|2013-10-03|
+----+-----------+-----------+---------+--------------------+----------+
```
The `Users` table holds all users. Each user has an unique Users_Id, and Role is an ENUM type of (‘client’, ‘driver’, ‘partner’).
```
+----------+--------+--------+
| Users_Id | Banned |  Role  |
+----------+--------+--------+
|    1     |   No   | client |
|    2     |   Yes  | client |
|    3     |   No   | client |
|    4     |   No   | client |
|    10    |   No   | driver |
|    11    |   No   | driver |
|    12    |   No   | driver |
|    13    |   No   | driver |
+----------+--------+--------+
```
Write a SQL query to find the cancellation rate of requests made by unbanned users (both client and driver must be unbanned) between Oct 1, 2013 and Oct 3, 2013. The cancellation rate is computed by dividing the number of canceled (by client or driver) requests made by unbanned users by the total number of requests made by unbanned users.

For the above tables, your SQL query should return the following rows with the cancellation rate being rounded to two decimal places.
```
+------------+-------------------+
|     Day    | Cancellation Rate |
+------------+-------------------+
| 2013-10-01 |       0.33        |
| 2013-10-02 |       0.00        |
| 2013-10-03 |       0.50        |
+------------+-------------------+
```


```mysql
select request_at as Day,
    round(count(if(status != 'completed', true, null)) / count(*), 2) as 'Cancellation Rate'
from trips
where client_id in (select users_id from users where Banned='No')
    and driver_id in (select users_id from users where Banned='No')
    and request_at between '2013-10-01' and '2013-10-03'
group by 1;
```

## 196. Delete Duplicate Emails (Easy)
Write a SQL query to delete all duplicate email entries in a table named `Person`, keeping only unique emails based on its smallest Id.
```
+----+------------------+
| Id | Email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
| 3  | john@example.com |
+----+------------------+
```
Id is the primary key column for this table.
For example, after running your query, the above `Person` table should have the following rows:
```
+----+------------------+
| Id | Email            |
+----+------------------+
| 1  | john@example.com |
| 2  | bob@example.com  |
+----+------------------+
```
Note:

Your output is the whole Person table after executing your sql. Use `delete` statement

```mysql
delete p1
from person p1, person p2
where p1.Email = p2.Email and p1.id > p2.id;
```



## 184. Department Highest Salary (Medium)
The `Employee` table holds all employees. Every employee has an Id, a salary, and there is also a column for the department Id.
```
+----+-------+--------+--------------+
| Id | Name  | Salary | DepartmentId |
+----+-------+--------+--------------+
| 1  | Joe   | 70000  | 1            |
| 2  | Jim   | 90000  | 1            |
| 3  | Henry | 80000  | 2            |
| 4  | Sam   | 60000  | 2            |
| 5  | Max   | 90000  | 1            |
+----+-------+--------+--------------+
```
The `Department` table holds all departments of the company.
```
+----+----------+
| Id | Name     |
+----+----------+
| 1  | IT       |
| 2  | Sales    |
+----+----------+
```
Write a SQL query to find employees who have the highest salary in each of the departments. For the above tables, your SQL query should return the following rows (order of rows does not matter).
```
+------------+----------+--------+
| Department | Employee | Salary |
+------------+----------+--------+
| IT         | Max      | 90000  |
| IT         | Jim      | 90000  |
| Sales      | Henry    | 80000  |
+------------+----------+--------+
```
Explanation:

Max and Jim both have the highest salary in the IT department and Henry has the highest salary in the Sales department.


```mysql
select d.name as 'Department', e.name as 'Employee', e.salary as 'Salary'
from employee e
join Department d on e.DepartmentId = d.id
where (e.DepartmentId, e.Salary) in (
    select DepartmentId, max(Salary)
    from employee
    group by DepartmentId
);
```


## 1179. Reformat Department Table (Easy)
Table: `Department`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| revenue       | int     |
| month         | varchar |
+---------------+---------+
```
(id, month) is the primary key of this table.
The table has information about the revenue of each department per month.
The month has values in ["Jan","Feb","Mar","Apr","May","Jun","Jul","Aug","Sep","Oct","Nov","Dec"].


Write an SQL query to reformat the table such that there is a department id column and a revenue column for each month.

The query result format is in the following example:

`Department` table:
```
+------+---------+-------+
| id   | revenue | month |
+------+---------+-------+
| 1    | 8000    | Jan   |
| 2    | 9000    | Jan   |
| 3    | 10000   | Feb   |
| 1    | 7000    | Feb   |
| 1    | 6000    | Mar   |
+------+---------+-------+
```
Result table:
```
+------+-------------+-------------+-------------+-----+-------------+
| id   | Jan_Revenue | Feb_Revenue | Mar_Revenue | ... | Dec_Revenue |
+------+-------------+-------------+-------------+-----+-------------+
| 1    | 8000        | 7000        | 6000        | ... | null        |
| 2    | 9000        | null        | null        | ... | null        |
| 3    | null        | 10000       | null        | ... | null        |
+------+-------------+-------------+-------------+-----+-------------+
```

Note that the result table has 13 columns (1 for the department id + 12 for the months).



```mysql
select
    id,
    max(if(month='jan', revenue, null)) as Jan_Revenue,
    max(if(month='Feb', revenue, null)) as Feb_Revenue,
    max(if(month='Mar', revenue, null)) as Mar_Revenue,
    max(if(month='Apr', revenue, null)) as Apr_Revenue,
    max(if(month='May', revenue, null)) as May_Revenue,
    max(if(month='Jun', revenue, null)) as Jun_Revenue,
    max(if(month='Jul', revenue, null)) as Jul_Revenue,
    max(if(month='Aug', revenue, null)) as Aug_Revenue,
    max(if(month='Sep', revenue, null)) as Sep_Revenue,
    max(if(month='Oct', revenue, null)) as Oct_Revenue,
    max(if(month='Nov', revenue, null)) as Nov_Revenue,
    max(if(month='Dec', revenue, null)) as Dec_Revenue
from department
group by id;
```


## 626. Exchange Seats (Medium)
Mary is a teacher in a middle school and she has a table seat storing students' names and their corresponding seat ids.

The column id is continuous increment.


Mary wants to change seats for the adjacent students.


Can you write a SQL query to output the result for Mary?

```
+---------+---------+
|    id   | student |
+---------+---------+
|    1    | Abbot   |
|    2    | Doris   |
|    3    | Emerson |
|    4    | Green   |
|    5    | Jeames  |
+---------+---------+
```
For the sample input, the output is:

```
+---------+---------+
|    id   | student |
+---------+---------+
|    1    | Doris   |
|    2    | Abbot   |
|    3    | Green   |
|    4    | Emerson |
|    5    | Jeames  |
+---------+---------+
```
Note:
If the number of students is odd, there is no need to change the last one's seat.



Solution1 (case):
```mysql
select
    (case
        when mod(id, 2) != 0 and counts != id then id+1
        when mod(id, 2) != 0 and counts = id then id
        else id -1
    end) id,
    student
from
    seat,
    (select count(*) as counts
    from seat) as seat_counts
order by id asc;
```

Solution2 (coalesce, bit manipulation):
```mysql
select s1.id, coalesce(s2.student, s1.student) as tudent
from seat s1
left join seat s2 on ((s1.id+1)^1)-1 = s2.id
order by s1.id asc;
````

## 180. Consecutive Numbers (Medium)
Write a SQL query to find all numbers that appear at least three times consecutively.
```
+----+-----+
| Id | Num |
+----+-----+
| 1  |  1  |
| 2  |  1  |
| 3  |  1  |
| 4  |  2  |
| 5  |  1  |
| 6  |  2  |
| 7  |  2  |
+----+-----+
```
For example, given the above `Logs` table, 1 is the only number that appears consecutively for at least three times.
```
+-----------------+
| ConsecutiveNums |
+-----------------+
| 1               |
+-----------------+
```



```mysql
select l1.num 'ConsecutiveNumbs'
from logs l1, logs l2, logs l3
where l1.id = l2.id-1
    and l2.id = l3.id-1
    and l1.num = l2.num
    and l2.num = l3.num;
```



## 615. Average Salary: Departments VS Company (Hard)
Given two tables as below, write a query to display the comparison result (higher/lower/same) of the average salary of employees in a department to the company's average salary.


Table: `salary`
```
| id | employee_id | amount | pay_date   |
|----|-------------|--------|------------|
| 1  | 1           | 9000   | 2017-03-31 |
| 2  | 2           | 6000   | 2017-03-31 |
| 3  | 3           | 10000  | 2017-03-31 |
| 4  | 1           | 7000   | 2017-02-28 |
| 5  | 2           | 6000   | 2017-02-28 |
| 6  | 3           | 8000   | 2017-02-28 |
```

The employee_id column refers to the employee_id in the following table `employee`.

```
| employee_id | department_id |
|-------------|---------------|
| 1           | 1             |
| 2           | 2             |
| 3           | 2             |
```

So for the sample data above, the result is:

```
| pay_month | department_id | comparison  |
|-----------|---------------|-------------|
| 2017-03   | 1             | higher      |
| 2017-03   | 2             | lower       |
| 2017-02   | 1             | same        |
| 2017-02   | 2             | same        |
```

```mysql
select d.pay_month, d.department_id,
    (case
        when d.department_avg > c.company_avg then 'higher'
        when d.department_avg < c.company_avg then 'lower'
        else 'same'
    end) comparison
from
    (select e.department_id, avg(s.amount) department_avg, date_format(s.pay_date, '%Y-%m') pay_month
    from salary s
    join employee e on s.employee_id = e.employee_id
    group by e.department_id, pay_month) d
join
    (select avg(amount) company_avg, date_format(pay_date, '%Y-%m') pay_month
    from salary
    group by pay_month) c
on d.pay_month = c.pay_month;
```



## 601. Human Traffic of Stadium (Hard)
Table: `Stadium`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| visit_date    | date    |
| people        | int     |
+---------------+---------+
```
visit_date is the primary key for this table.
Each row of this table contains the visit date and visit id to the stadium with the number of people during the visit.
No two rows will have the same visit_date, and as the id increases, the dates increase as well.


Write an SQL query to display the records with three or more rows with consecutive id's, and the number of people is greater than or equal to 100 for each.

Return the result table ordered by visit_date in ascending order.

The query result format is in the following example.



`Stadium` table:
```
+------+------------+-----------+
| id   | visit_date | people    |
+------+------------+-----------+
| 1    | 2017-01-01 | 10        |
| 2    | 2017-01-02 | 109       |
| 3    | 2017-01-03 | 150       |
| 4    | 2017-01-04 | 99        |
| 5    | 2017-01-05 | 145       |
| 6    | 2017-01-06 | 1455      |
| 7    | 2017-01-07 | 199       |
| 8    | 2017-01-09 | 188       |
+------+------------+-----------+
```

Result table:
```
+------+------------+-----------+
| id   | visit_date | people    |
+------+------------+-----------+
| 5    | 2017-01-05 | 145       |
| 6    | 2017-01-06 | 1455      |
| 7    | 2017-01-07 | 199       |
| 8    | 2017-01-09 | 188       |
+------+------------+-----------+
```
The four rows with ids 5, 6, 7, and 8 have consecutive ids and each of them has >= 100 people attended. Note that row 8 was included even though the visit_date was not the next day after row 7.
The rows with ids 2 and 3 are not included because we need at least three consecutive ids.




```mysql
select distinct t1.*
from stadium t1, stadium t2, stadium t3
where t1.people >= 100 and t2.people >= 100 and t3.people>=100
    and (  -- only the order of t1 is considered
            (t1.id = t2.id - 1 and t1.id = t3.id - 2) -- t1 < t2 < t3
        or
            (t3.id = t1.id - 1 and t3.id = t2.id - 2) -- t3 < t1 < t2
        or
            (t3.id = t2.id - 1 and t3.id = t1.id - 2) -- t3 < t2 < t1
        )
order by t1.id;
```



## 1270. All People Report to the Given Manager (Medium)
Table: `Employees`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| employee_id   | int     |
| employee_name | varchar |
| manager_id    | int     |
+---------------+---------+
```
employee_id is the primary key for this table.
Each row of this table indicates that the employee with ID employee_id and name employee_name reports his work to his/her direct manager with manager_id
The head of the company is the employee with employee_id = 1.


Write an SQL query to find employee_id of all employees that directly or indirectly report their work to the head of the company.

The indirect relation between managers will not exceed 3 managers as the company is small.

Return result table in any order without duplicates.

The query result format is in the following example:



```mysql
select e1.employee_id
from employees e1, employees e2, employees e3
where e1.manager_id = e2.employee_id
    and e2.manager_id = e3.employee_id
    and e3.manager_id = 1
    and e1.employee_id != 1;
```



## 1384. Total Sales Amount by Year (Hard)
Table: `Product`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| product_id    | int     |
| product_name  | varchar |
+---------------+---------+
```
product_id is the primary key for this table.
product_name is the name of the product.


Table: `Sales`
```
+---------------------+---------+
| Column Name         | Type    |
+---------------------+---------+
| product_id          | int     |
| period_start        | varchar |
| period_end          | date    |
| average_daily_sales | int     |
+---------------------+---------+
```
product_id is the primary key for this table.
period_start and period_end indicates the start and end date for sales period, both dates are inclusive.
The average_daily_sales column holds the average daily sales amount of the items for the period.

Write an SQL query to report the Total sales amount of each item for each year, with corresponding product name, product_id, product_name and report_year.

Dates of the sales years are between 2018 to 2020. Return the result table ordered by product_id and report_year.

The query result format is in the following example:


`Product` table:
```
+------------+--------------+
| product_id | product_name |
+------------+--------------+
| 1          | LC Phone     |
| 2          | LC T-Shirt   |
| 3          | LC Keychain  |
+------------+--------------+
```

`Sales` table:
```
+------------+--------------+-------------+---------------------+
| product_id | period_start | period_end  | average_daily_sales |
+------------+--------------+-------------+---------------------+
| 1          | 2019-01-25   | 2019-02-28  | 100                 |
| 2          | 2018-12-01   | 2020-01-01  | 10                  |
| 3          | 2019-12-01   | 2020-01-31  | 1                   |
+------------+--------------+-------------+---------------------+
```
Result table:
```
+------------+--------------+-------------+--------------+
| product_id | product_name | report_year | total_amount |
+------------+--------------+-------------+--------------+
| 1          | LC Phone     |    2019     | 3500         |
| 2          | LC T-Shirt   |    2018     | 310          |
| 2          | LC T-Shirt   |    2019     | 3650         |
| 2          | LC T-Shirt   |    2020     | 10           |
| 3          | LC Keychain  |    2019     | 31           |
| 3          | LC Keychain  |    2020     | 31           |
+------------+--------------+-------------+--------------+
```
LC Phone was sold for the period of 2019-01-25 to 2019-02-28, and there are 35 days for this period. Total amount 35*100 = 3500.
LC T-shirt was sold for the period of 2018-12-01 to 2020-01-01, and there are 31, 365, 1 days for years 2018, 2019 and 2020 respectively.
LC Keychain was sold for the period of 2019-12-01 to 2020-01-31, and there are 31, 31 days for years 2019 and 2020 respectively.



```mysql
select a.product_id, b.product_name, a.report_year, a.total_amount
from (
    select product_id, '2018' as report_year,
        average_daily_sales * (datediff(least(period_end, '2018-12-31'), greatest(period_start, '2018-01-01'))+1) as total_amount
    from sales
    where year(period_start)=2018 or year(period_end)=2018

    union all

    select product_id, '2019' as report_year,
        average_daily_sales * (datediff(least(period_end, '2019-12-31'), greatest(period_start, '2019-01-01'))+1) as total_amount
    from sales
    where year(period_start)<=2019 and year(period_end)>=2019 -- becareful!! <= & >=

    union all

    select product_id, '2020' as report_year,
        average_daily_sales * (datediff(least(period_end, '2020-12-31'), greatest(period_start, '2020-01-01'))+1) as total_amount
    from sales
    where year(period_start)=2020 or year(period_end)=2020
) a
left join product b
on a.product_id = b.product_id
order by a.product_id, a.report_year;
```



## 182. Duplicate Emails (Easy)
Write a SQL query to find all duplicate emails in a table named `Person`.
```
+----+---------+
| Id | Email   |
+----+---------+
| 1  | a@b.com |
| 2  | c@d.com |
| 3  | a@b.com |
+----+---------+
```
For example, your query should return the following for the above table:
```
+---------+
| Email   |
+---------+
| a@b.com |
+---------+
```
Note: All emails are in lowercase.

```mysql
select email
from person
group by email
having count(email) > 1;
```

## 1412. Find the Quiet Students in All Exams (Hard)
Table: `Student`
```
+---------------------+---------+
| Column Name         | Type    |
+---------------------+---------+
| student_id          | int     |
| student_name        | varchar |
+---------------------+---------+
```
student_id is the primary key for this table.
student_name is the name of the student.


Table: `Exam`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| exam_id       | int     |
| student_id    | int     |
| score         | int     |
+---------------+---------+
```
(exam_id, student_id) is the primary key for this table.
Student with student_id got score points in exam with id exam_id.


A "quite" student is the one who took at least one exam and didn't score neither the high score nor the low score.

Write an SQL query to report the students (student_id, student_name) being "quiet" in ALL exams.

Don't return the student who has never taken any exam. Return the result table ordered by student_id.

The query result format is in the following example.



Student table:
```
+-------------+---------------+
| student_id  | student_name  |
+-------------+---------------+
| 1           | Daniel        |
| 2           | Jade          |
| 3           | Stella        |
| 4           | Jonathan      |
| 5           | Will          |
+-------------+---------------+
```
`Exam` table:
```
+------------+--------------+-----------+
| exam_id    | student_id   | score     |
+------------+--------------+-----------+
| 10         |     1        |    70     |
| 10         |     2        |    80     |
| 10         |     3        |    90     |
| 20         |     1        |    80     |
| 30         |     1        |    70     |
| 30         |     3        |    80     |
| 30         |     4        |    90     |
| 40         |     1        |    60     |
| 40         |     2        |    70     |
| 40         |     4        |    80     |
+------------+--------------+-----------+
```

Result table:
```
+-------------+---------------+
| student_id  | student_name  |
+-------------+---------------+
| 2           | Jade          |
+-------------+---------------+
```

For exam 1: Student 1 and 3 hold the lowest and high score respectively.
For exam 2: Student 1 hold both highest and lowest score.
For exam 3 and 4: Studnet 1 and 4 hold the lowest and high score respectively.
Student 2 and 5 have never got the highest or lowest in any of the exam.
Since student 5 is not taking any exam, he is excluded from the result.
So, we only return the information of Student 2.


```mysql
with cte as (
    select e.exam_id, e.student_id, s.student_name,
        rank() over (partition by exam_id order by score) rank_asc,
        rank() over (partition by exam_id order by score desc) rank_desc
    from exam e
    left join student s
        on e.student_id = s.student_id
)
select distinct student_id, student_name
from cte
where student_id not in (
    select student_id
    from cte
    where rank_asc=1 or rank_desc=1)
order by student_id;
```

## 627. Swap Salary (Easy)
Given a table `salary`, such as the one below, that has m=male and f=female values. Swap all f and m values (i.e., change all f values to m and vice versa) with a single update statement and no intermediate temp table.

Note that you must write a single update statement, DO NOT write any select statement for this problem.



Example:
```
| id | name | sex | salary |
|----|------|-----|--------|
| 1  | A    | m   | 2500   |
| 2  | B    | f   | 1500   |
| 3  | C    | m   | 5500   |
| 4  | D    | f   | 500    |
```
After running your update statement, the above salary table should have the following rows:
```
| id | name | sex | salary |
|----|------|-----|--------|
| 1  | A    | f   | 2500   |
| 2  | B    | m   | 1500   |
| 3  | C    | f   | 5500   |
| 4  | D    | m   | 500    |
```



```mysql
update salary
set
    sex = (case sex
        when 'm' then 'f'
        else 'm'
        end);
```

```mysql
update salary
set
    sex = if(sex='m', 'f', 'm');
```


## 1355. Activity Participants (Medium)
Table: `Friends`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| name          | varchar |
| activity      | varchar |
+---------------+---------+
```
id is the id of the friend and primary key for this table.
name is the name of the friend.
activity is the name of the activity which the friend takes part in.

Table: `Activities`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| name          | varchar |
+---------------+---------+
```
id is the primary key for this table.
name is the name of the activity.


Write an SQL query to find the names of all the activities with neither maximum, nor minimum number of participants.

Return the result table in any order. Each activity in table Activities is performed by any person in the table Friends.

The query result format is in the following example:

`Friends` table:
```
+------+--------------+---------------+
| id   | name         | activity      |
+------+--------------+---------------+
| 1    | Jonathan D.  | Eating        |
| 2    | Jade W.      | Singing       |
| 3    | Victor J.    | Singing       |
| 4    | Elvis Q.     | Eating        |
| 5    | Daniel A.    | Eating        |
| 6    | Bob B.       | Horse Riding  |
+------+--------------+---------------+
```
`Activities` table:
```
+------+--------------+
| id   | name         |
+------+--------------+
| 1    | Eating       |
| 2    | Singing      |
| 3    | Horse Riding |
+------+--------------+
```

Result table:
```
+--------------+
| activity     |
+--------------+
| Singing      |
+--------------+
```
Eating activity is performed by 3 friends, maximum number of participants, (Jonathan D. , Elvis Q. and Daniel A.)
Horse Riding activity is performed by 1 friend, minimum number of participants, (Bob B.)
Singing is performed by 2 friends (Victor J. and Jade W.)



```mysql
select activity
from
    (select activity,
        rank() over(order by count(*)) as cnt_asc,
        rank() over(order by count(*) desc) as cnt_desc
    from friends
    group by activity) a
where
    a.cnt_asc != 1 and a.cnt_desc != 1;
```



## 1132. Reported Posts II (Medium)
Table: `Actions`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | int     |
| post_id       | int     |
| action_date   | date    |
| action        | enum    |
| extra         | varchar |
+---------------+---------+
```
There is no primary key for this table, it may have duplicate rows.
The action column is an ENUM type of ('view', 'like', 'reaction', 'comment', 'report', 'share').
The extra column has optional information about the action such as a reason for report or a type of reaction.

Table: `Removals`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| post_id       | int     |
| remove_date   | date    |
+---------------+---------+
```
post_id is the primary key of this table.
Each row in this table indicates that some post was removed as a result of being reported or as a result of an admin review.


Write an SQL query to find the average for daily percentage of posts that got removed after being reported as spam, rounded to 2 decimal places.

The query result format is in the following example:

`Actions` table:
```
+---------+---------+-------------+--------+--------+
| user_id | post_id | action_date | action | extra  |
+---------+---------+-------------+--------+--------+
| 1       | 1       | 2019-07-01  | view   | null   |
| 1       | 1       | 2019-07-01  | like   | null   |
| 1       | 1       | 2019-07-01  | share  | null   |
| 2       | 2       | 2019-07-04  | view   | null   |
| 2       | 2       | 2019-07-04  | report | spam   |
| 3       | 4       | 2019-07-04  | view   | null   |
| 3       | 4       | 2019-07-04  | report | spam   |
| 4       | 3       | 2019-07-02  | view   | null   |
| 4       | 3       | 2019-07-02  | report | spam   |
| 5       | 2       | 2019-07-03  | view   | null   |
| 5       | 2       | 2019-07-03  | report | racism |
| 5       | 5       | 2019-07-03  | view   | null   |
| 5       | 5       | 2019-07-03  | report | racism |
+---------+---------+-------------+--------+--------+
```
`Removals` table:
```
+---------+-------------+
| post_id | remove_date |
+---------+-------------+
| 2       | 2019-07-20  |
| 3       | 2019-07-18  |
+---------+-------------+
```
Result table:
```
+-----------------------+
| average_daily_percent |
+-----------------------+
| 75.00                 |
+-----------------------+
```
The percentage for 2019-07-04 is 50% because only one post of two spam reported posts was removed.
The percentage for 2019-07-02 is 100% because one post was reported as spam and it was removed.
The other days had no spam reports so the average is (50 + 100) / 2 = 75%
Note that the output is only one number and that we do not care about the remove dates.


```mysql
select round(avg(percent), 2) average_daily_percent
from (
    select a.action_date,
        count(distinct r.post_id)/count(distinct a.post_id)*100 as percent
    from actions a
    left join removals r
    on a.post_id = r.post_id
    where a.extra = 'spam'
    group by a.action_date) tmp;
```

## 1517. Find Users with Valid E-Mails (Easy)
Table: `Users`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| user_id       | int     |
| name          | varchar |
| mail          | varchar |
+---------------+---------+
```
user_id is the primary key for this table.
This table contains information of the users signed up in a website. Some e-mails are invalid.


Write an SQL query to find the users who have valid emails.

A valid e-mail has a prefix name and a domain where:

The prefix name is a string that may contain letters (upper or lower case), digits, underscore '_', period '.' and/or dash '-'. The prefix name must start with a letter.
The domain is '@leetcode.com'.
Return the result table in any order.

The query result format is in the following example.



`Users`
```
+---------+-----------+-------------------------+
| user_id | name      | mail                    |
+---------+-----------+-------------------------+
| 1       | Winston   | winston@leetcode.com    |
| 2       | Jonathan  | jonathanisgreat         |
| 3       | Annabelle | bella-@leetcode.com     |
| 4       | Sally     | sally.come@leetcode.com |
| 5       | Marwan    | quarz#2020@leetcode.com |
| 6       | David     | david69@gmail.com       |
| 7       | Shapiro   | .shapo@leetcode.com     |
+---------+-----------+-------------------------+
```
`Result` table:
```
+---------+-----------+-------------------------+
| user_id | name      | mail                    |
+---------+-----------+-------------------------+
| 1       | Winston   | winston@leetcode.com    |
| 3       | Annabelle | bella-@leetcode.com     |
| 4       | Sally     | sally.come@leetcode.com |
+---------+-----------+-------------------------+
```
The mail of user 2 doesn't have a domain.
The mail of user 5 has # sign which is not allowed.
The mail of user 6 doesn't have leetcode domain.
The mail of user 7 starts with a period.



A detailed explanation of the following regular expression solution:
```
'^[A-Za-z]+[A-Za-z0-9\_\.\-]*@leetcode.com$'
```
1. ^ means the beginning of the string
 \- This is important because without it, we can have something like `.shapiro@leetcode.com`
 This is because *part* of the regex matches the pattern perfectly.
 The part that is `shapiro@leetcode.com`.
 This is how I understand it: regex will return the whole thing as long as part of it matches.
 By adding ^ we are saying: you have to ma match FROM THE START.

2. [] means character set. [A-Z] means any upper case chars. In other words, the short dash in the character set means range.

3. After the first and second set, there is a notation: + or *.
 \+ means at least one of the character from the preceding charset, and * means 0 or more.

4. \ inside the charset mean skipping. In other words, \. means we want to the dot as it is. Remember, for example, \- means range in the character set. So what if we would like to find - itself as a character? use \\-.

5. Everything else, like `@leetcode\.com$` refers to exact match. the $ sign indicateing "end with"
```mysql
select *
from users
where mail regexp '^[A-Za-z][A-Za-z0-9\_\.\-]*@leetcode\.com$';
```


## 1454. Active Users (Medium)
Table `Accounts`:
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| name          | varchar |
+---------------+---------+
```
the id is the primary key for this table.
This table contains the account id and the user name of each account.


Table `Logins`:
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| login_date    | date    |
+---------------+---------+
```
There is no primary key for this table, it may contain duplicates.
This table contains the account id of the user who logged in and the login date. A user may log in multiple times in the day.


Write an SQL query to find the id and the name of active users.

Active users are those who logged in to their accounts for 5 or more consecutive days.

Return the result table ordered by the id.

The query result format is in the following example:

`Accounts` table:
```
+----+----------+
| id | name     |
+----+----------+
| 1  | Winston  |
| 7  | Jonathan |
+----+----------+
```
`Logins` table:
```
+----+------------+
| id | login_date |
+----+------------+
| 7  | 2020-05-30 |
| 1  | 2020-05-30 |
| 7  | 2020-05-31 |
| 7  | 2020-06-01 |
| 7  | 2020-06-02 |
| 7  | 2020-06-02 |
| 7  | 2020-06-03 |
| 1  | 2020-06-07 |
| 7  | 2020-06-10 |
+----+------------+
```
Result table:
```
+----+----------+
| id | name     |
+----+----------+
| 7  | Jonathan |
+----+----------+
```
User Winston with id = 1 logged in 2 times only in 2 different days, so, Winston is not an active user.
User Jonathan with id = 7 logged in 7 times in 6 different days, five of them were consecutive days, so, Jonathan is an active user.



```mysql
select distinct l1.id,
    (select name from accounts where id=l1.id) name
from logins l1, logins l2
where l1.id = l2.id
    and datediff(l1.login_date, l2.login_date) between 1 and 4
group by l1.id, l1.login_date
having count(distinct l2.login_date)=4;
```

## 197. Rising Temperature (Easy)
Table: `Weather`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| id            | int     |
| recordDate    | date    |
| temperature   | int     |
+---------------+---------+
```
id is the primary key for this table.
This table contains information about the temperature in a certain day.


Write an SQL query to find all dates' id with higher temperature compared to its previous dates (yesterday).

Return the result table in any order.

The query result format is in the following example:

`Weather`
```
+----+------------+-------------+
| id | recordDate | Temperature |
+----+------------+-------------+
| 1  | 2015-01-01 | 10          |
| 2  | 2015-01-02 | 25          |
| 3  | 2015-01-03 | 20          |
| 4  | 2015-01-04 | 30          |
+----+------------+-------------+
```
Result table:
```
+----+
| id |
+----+
| 2  |
| 4  |
+----+
```
In 2015-01-02, temperature was higher than the previous day (10 -> 25).
In 2015-01-04, temperature was higher than the previous day (30 -> 20).



```mysql
select w1.id
from weather w1, weather w2
where datediff(w1.recorddate, w2.recorddate) = 1
    and w1.temperature > w2.temperature;
```

## 1479. Sales by Day of the Week (Hard)
Table: `Orders`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| order_id      | int     |
| customer_id   | int     |
| order_date    | date    |
| item_id       | varchar |
| quantity      | int     |
+---------------+---------+
```
(ordered_id, item_id) is the primary key for this table.
This table contains information of the orders placed.
order_date is the date when item_id was ordered by the customer with id customer_id.


Table: `Items`
```
+---------------------+---------+
| Column Name         | Type    |
+---------------------+---------+
| item_id             | varchar |
| item_name           | varchar |
| item_category       | varchar |
+---------------------+---------+
```
item_id is the primary key for this table.
item_name is the name of the item.
item_category is the category of the item.


You are the business owner and would like to obtain a sales report for category items and day of the week.

Write an SQL query to report how many units in each category have been ordered on each day of the week.

Return the result table ordered by category.

The query result format is in the following example:



`Orders` table:
```
+------------+--------------+-------------+--------------+-------------+
| order_id   | customer_id  | order_date  | item_id      | quantity    |
+------------+--------------+-------------+--------------+-------------+
| 1          | 1            | 2020-06-01  | 1            | 10          |
| 2          | 1            | 2020-06-08  | 2            | 10          |
| 3          | 2            | 2020-06-02  | 1            | 5           |
| 4          | 3            | 2020-06-03  | 3            | 5           |
| 5          | 4            | 2020-06-04  | 4            | 1           |
| 6          | 4            | 2020-06-05  | 5            | 5           |
| 7          | 5            | 2020-06-05  | 1            | 10          |
| 8          | 5            | 2020-06-14  | 4            | 5           |
| 9          | 5            | 2020-06-21  | 3            | 5           |
+------------+--------------+-------------+--------------+-------------+
```

`Items` table:
```
+------------+----------------+---------------+
| item_id    | item_name      | item_category |
+------------+----------------+---------------+
| 1          | LC Alg. Book   | Book          |
| 2          | LC DB. Book    | Book          |
| 3          | LC SmarthPhone | Phone         |
| 4          | LC Phone 2020  | Phone         |
| 5          | LC SmartGlass  | Glasses       |
| 6          | LC T-Shirt XL  | T-Shirt       |
+------------+----------------+---------------+
```
Result table:
```
+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
| Category   | Monday    | Tuesday   | Wednesday | Thursday  | Friday    | Saturday  | Sunday    |
+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
| Book       | 20        | 5         | 0         | 0         | 10        | 0         | 0         |
| Glasses    | 0         | 0         | 0         | 0         | 5         | 0         | 0         |
| Phone      | 0         | 0         | 5         | 1         | 0         | 0         | 10        |
| T-Shirt    | 0         | 0         | 0         | 0         | 0         | 0         | 0         |
+------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
```
On Monday (2020-06-01, 2020-06-08) were sold a total of 20 units (10 + 10) in the category Book (ids: 1, 2).
On Tuesday (2020-06-02) were sold a total of 5 units  in the category Book (ids: 1, 2).
On Wednesday (2020-06-03) were sold a total of 5 units in the category Phone (ids: 3, 4).
On Thursday (2020-06-04) were sold a total of 1 unit in the category Phone (ids: 3, 4).
On Friday (2020-06-05) were sold 10 units in the category Book (ids: 1, 2) and 5 units in Glasses (ids: 5).
On Saturday there are no items sold.
On Sunday (2020-06-14, 2020-06-21) were sold a total of 10 units (5 +5) in the category Phone (ids: 3, 4).
There are no sales of T-Shirt.



```mysql
select i.item_category,
    sum(case when dayofweek(o.order_date)=2 then o.quantity else 0 end) Monday,
    sum(case when dayofweek(o.order_date)=3 then o.quantity else 0 end) Tuesday,
    sum(case when dayofweek(o.order_date)=4 then o.quantity else 0 end) Wednesday,
    sum(case when dayofweek(o.order_date)=5 then o.quantity else 0 end) Thursday,
    sum(case when dayofweek(o.order_date)=6 then o.quantity else 0 end) Friday,
    sum(case when dayofweek(o.order_date)=7 then o.quantity else 0 end) Saturday,
    sum(case when dayofweek(o.order_date)=1 then o.quantity else 0 end) Sunday
from item i
    left join orders o
        on i.item_id = o.item_id
group by 1
order by 1;
```



## 183. Customers Who Never Order (Easy)
Suppose that a website contains two tables, the `Customers` table and the `Orders` table. Write a SQL query to find all customers who never order anything.

Table: `Customers`.
```
+----+-------+
| Id | Name  |
+----+-------+
| 1  | Joe   |
| 2  | Henry |
| 3  | Sam   |
| 4  | Max   |
+----+-------+
```
Table: `Orders`.
```
+----+------------+
| Id | CustomerId |
+----+------------+
| 1  | 3          |
| 2  | 1          |
+----+------------+
```
Using the above tables as example, return the following:
```
+-----------+
| Customers |
+-----------+
| Henry     |
| Max       |
+-----------+
```



solution 1:
```mysql
select c.name as customers
from customers
where c.id not in (
    select distinct customerid
    from orders);
```
solution 2:
```mysql
select c.name as customers
from customers c
    left join orders o
        on c.id = o.customerid
where o.customerid is null;
```


## 569. Median Employee Salary (Hard)
The `Employee` table holds all employees. The employee table has three columns: Employee Id, Company Name, and Salary.
```
+-----+------------+--------+
|Id   | Company    | Salary |
+-----+------------+--------+
|1    | A          | 2341   |
|2    | A          | 341    |
|3    | A          | 15     |
|4    | A          | 15314  |
|5    | A          | 451    |
|6    | A          | 513    |
|7    | B          | 15     |
|8    | B          | 13     |
|9    | B          | 1154   |
|10   | B          | 1345   |
|11   | B          | 1221   |
|12   | B          | 234    |
|13   | C          | 2345   |
|14   | C          | 2645   |
|15   | C          | 2645   |
|16   | C          | 2652   |
|17   | C          | 65     |
+-----+------------+--------+
```
Write a SQL query to find the median salary of each company. Bonus points if you can solve it without using any built-in SQL functions.
```
+-----+------------+--------+
|Id   | Company    | Salary |
+-----+------------+--------+
|5    | A          | 451    |
|6    | A          | 513    |
|12   | B          | 234    |
|9    | B          | 1154   |
|14   | C          | 2645   |
+-----+------------+--------+
```



```mysql
select rankingtable.id, rankingtable.company, rankingtable.salary
from
    (SELECT
            e.Id,
            e.Salary,
            e.Company,
            IF(@prev = e.Company, @ranking:=@ranking + 1, @ranking:=1) AS ranking,
            @prev:=e.Company
    FROM
        Employee e, (SELECT @ranking:=0, @prev:=0) AS temp
    ORDER BY e.Company , e.Salary , e.Id) rankingtable
inner join
    (select
        e2.company, count(*) totalcount
     from employee e2
     group by e2.company) companycount
 on companycount.company = rankingtable.company
 where
    ranking = floor((totalcount+1)/2)
    or ranking = floor((totalcount+2)/2);
```



## 610. Triangle Judgement (Easy)
A pupil Tim gets homework to identify whether three line segments could possibly form a triangle.


However, this assignment is very heavy because there are hundreds of records to calculate.


Could you help Tim by writing a query to judge whether these three sides can form a triangle, assuming table `triangle` holds the length of the three sides x, y and z.

```
| x  | y  | z  |
|----|----|----|
| 13 | 15 | 30 |
| 10 | 20 | 15 |
```
For the sample data above, your query should return the follow result:
```
| x  | y  | z  | triangle |
|----|----|----|----------|
| 13 | 15 | 30 | No       |
| 10 | 20 | 15 | Yes      |
```

```mysql
select x, y, z,
    case
        when x + y > z and y + z > x and z + x > y then "Yes"
        else "No"
    end as "triangle"
from triangle;
```

## 1127. User Purchase Platform (Hard)
Table: `Spending`
```
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| user_id     | int     |
| spend_date  | date    |
| platform    | enum    |
| amount      | int     |
+-------------+---------+
```
The table logs the spendings history of users that make purchases from an online shopping website which has a desktop and a mobile application.
(user_id, spend_date, platform) is the primary key of this table.
The platform column is an ENUM type of ('desktop', 'mobile').
Write an SQL query to find the total number of users and the total amount spent using mobile only, desktop only and both mobile and desktop together for each date.

The query result format is in the following example:

`Spending` table:
```
+---------+------------+----------+--------+
| user_id | spend_date | platform | amount |
+---------+------------+----------+--------+
| 1       | 2019-07-01 | mobile   | 100    |
| 1       | 2019-07-01 | desktop  | 100    |
| 2       | 2019-07-01 | mobile   | 100    |
| 2       | 2019-07-02 | mobile   | 100    |
| 3       | 2019-07-01 | desktop  | 100    |
| 3       | 2019-07-02 | desktop  | 100    |
+---------+------------+----------+--------+
```

Result table:
```
+------------+----------+--------------+-------------+
| spend_date | platform | total_amount | total_users |
+------------+----------+--------------+-------------+
| 2019-07-01 | desktop  | 100          | 1           |
| 2019-07-01 | mobile   | 100          | 1           |
| 2019-07-01 | both     | 200          | 1           |
| 2019-07-02 | desktop  | 100          | 1           |
| 2019-07-02 | mobile   | 100          | 1           |
| 2019-07-02 | both     | 0            | 0           |
+------------+----------+--------------+-------------+
```
On 2019-07-01, user 1 purchased using both desktop and mobile, user 2 purchased using mobile only and user 3 purchased using desktop only.
On 2019-07-02, user 2 purchased using mobile only, user 3 purchased using desktop only and no one purchased using both platforms.



```mysql
select
    p.spend_date,
    p.platform,
    ifnull(sum(tmp.amount), 0) total_amount,
    count(tmp.user_id) total_users
from (
    select distinct (spend_date), 'desktop' platform from spending
    union
    select distinct (spend_date), 'mobile' platform from spending
    union
    select distinct (spend_date), 'both' platform from spending) p
left join (
    select
        spend_date,
        user_id,
        if(mobile_amount > 0, if(desktop_amount >0, 'both', 'mobile'), 'desktop') platform,
        (mobile_amount + desktop_amount) amount
    from
        (select
        spend_date,
        user_id,
        sum(case when platform='mobile' then amount else 0 end) mobile_amount,
        sum(case when platform='desktop' then amount else 0 end) desktop_amount
        from spending
        group by spend_date, user_id) o
    ) tmp
on p.spend_date = tmp.spend_date and p.platform = tmp.platform
group by p.spend_date, p.platform;
```



## 1212. Team Scores in Football Tournament (Medium)
Table: `Teams`
```
+---------------+----------+
| Column Name   | Type     |
+---------------+----------+
| team_id       | int      |
| team_name     | varchar  |
+---------------+----------+
```
team_id is the primary key of this table.
Each row of this table represents a single football team.
Table: `Matches`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| match_id      | int     |
| host_team     | int     |
| guest_team    | int     |
| host_goals    | int     |
| guest_goals   | int     |
+---------------+---------+
```
match_id is the primary key of this table.
Each row is a record of a finished match between two different teams.
Teams host_team and guest_team are represented by their IDs in the teams table (team_id) and they scored host_goals and guest_goals goals respectively.


You would like to compute the scores of all teams after all matches. Points are awarded as follows:
A team receives three points if they win a match (Score strictly more goals than the opponent team).
A team receives one point if they draw a match (Same number of goals as the opponent team).
A team receives no points if they lose a match (Score less goals than the opponent team).
Write an SQL query that selects the team_id, team_name and num_points of each team in the tournament after all described matches. Result table should be ordered by num_points (decreasing order). In case of a tie, order the records by team_id (increasing order).

The query result format is in the following example:

`Teams` table:
```
+-----------+--------------+
| team_id   | team_name    |
+-----------+--------------+
| 10        | Leetcode FC  |
| 20        | NewYork FC   |
| 30        | Atlanta FC   |
| 40        | Chicago FC   |
| 50        | Toronto FC   |
+-----------+--------------+
```

`Matches` table:
```
+------------+--------------+---------------+-------------+--------------+
| match_id   | host_team    | guest_team    | host_goals  | guest_goals  |
+------------+--------------+---------------+-------------+--------------+
| 1          | 10           | 20            | 3           | 0            |
| 2          | 30           | 10            | 2           | 2            |
| 3          | 10           | 50            | 5           | 1            |
| 4          | 20           | 30            | 1           | 0            |
| 5          | 50           | 30            | 1           | 0            |
+------------+--------------+---------------+-------------+--------------+
```

Result table:
```
+------------+--------------+---------------+
| team_id    | team_name    | num_points    |
+------------+--------------+---------------+
| 10         | Leetcode FC  | 7             |
| 20         | NewYork FC   | 3             |
| 50         | Toronto FC   | 3             |
| 30         | Atlanta FC   | 1             |
| 40         | Chicago FC   | 0             |
+------------+--------------+---------------+
```


```mysql
select t.team_id, t.team_name,
    sum(case when t.team_id = m.host_team and host_goals > guest_goals then 3
            when t.team_id = m.guest_team and guest_goals > host_goals then 3
            when host_goals = guest_goals then 1
            else 0
        end) num_points
from teams t
left join matches m
    on t.team_id = m.host_team or t.team_id = m.guest_team
group by 1, 2
order by 3 desc, 1 asc;
```



## 1398. Customers Who Bought Products A and B but Not C (Medium)
Table: `Customers`
```
+---------------------+---------+
| Column Name         | Type    |
+---------------------+---------+
| customer_id         | int     |
| customer_name       | varchar |
+---------------------+---------+
```
customer_id is the primary key for this table.
customer_name is the name of the customer.


Table: `Orders`
```
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| order_id      | int     |
| customer_id   | int     |
| product_name  | varchar |
+---------------+---------+
```
order_id is the primary key for this table.
customer_id is the id of the customer who bought the product "product_name".


Write an SQL query to report the customer_id and customer_name of customers who bought products "A", "B" but did not buy the product "C" since we want to recommend them buy this product.

Return the result table ordered by customer_id.

The query result format is in the following example.



`Customers` table:
```
+-------------+---------------+
| customer_id | customer_name |
+-------------+---------------+
| 1           | Daniel        |
| 2           | Diana         |
| 3           | Elizabeth     |
| 4           | Jhon          |
+-------------+---------------+
```
`Orders` table:
```
+------------+--------------+---------------+
| order_id   | customer_id  | product_name  |
+------------+--------------+---------------+
| 10         |     1        |     A         |
| 20         |     1        |     B         |
| 30         |     1        |     D         |
| 40         |     1        |     C         |
| 50         |     2        |     A         |
| 60         |     3        |     A         |
| 70         |     3        |     B         |
| 80         |     3        |     D         |
| 90         |     4        |     C         |
+------------+--------------+---------------+
```

Result table:
```
+-------------+---------------+
| customer_id | customer_name |
+-------------+---------------+
| 3           | Elizabeth     |
+-------------+---------------+
```
Only the customer_id with id 3 bought the product A and B but not the product C.


Solution 1:
```mysql
select counts.customer_id customer_id, customers.customer_name customer_name
from
    (select customer_id,
        sum(product_name = 'A') num_a,
        sum(product_name = 'B') num_b,
        sum(product_name = 'C') num_c
    from orders
    group by customer_id) counts
left join customers
    on customers.customer_id = counts.customer_id
where counts.num_a > 0 and counts.num_b > 0 and counts.num_c = 0
order by 1;
```

Solution 2 (faster):
```mysql
select distinct customer_id, customer_name
from customers
where custoemr_id in (
        select customer_id
        from orders
        where product_name = 'A')
    and customer_id in (
        select customer_id
        from orders
        where product_name = 'B')
    and customer_id not in (
        select customer_id
        from orders
        where product_name = 'C');
```

## 1435. Create a Session Bar Char (Easy)
Table: `Sessions`
```
+---------------------+---------+
| Column Name         | Type    |
+---------------------+---------+
| session_id          | int     |
| duration            | int     |
+---------------------+---------+
```
session_id is the primary key for this table.
duration is the time in seconds that a user has visited the application.


You want to know how long a user visits your application. You decided to create bins of "[0-5>", "[5-10>", "[10-15>" and "15 minutes or more" and count the number of sessions on it.

Write an SQL query to report the (bin, total) in any order.

The query result format is in the following example.

`Sessions` table:
```
+-------------+---------------+
| session_id  | duration      |
+-------------+---------------+
| 1           | 30            |
| 2           | 199           |
| 3           | 299           |
| 4           | 580           |
| 5           | 1000          |
+-------------+---------------+
```
Result table:
```
+--------------+--------------+
| bin          | total        |
+--------------+--------------+
| [0-5>        | 3            |
| [5-10>       | 1            |
| [10-15>      | 0            |
| 15 or more   | 1            |
+--------------+--------------+
```

For session_id 1, 2 and 3 have a duration greater or equal than 0 minutes and less than 5 minutes.
For session_id 4 has a duration greater or equal than 5 minutes and less than 10 minutes.
There are no session with a duration greater or equial than 10 minutes and less than 15 minutes.
For session_id 5 has a duration greater or equal than 15 minutes.



```mysql
select "[0-5>" bin, sum(duration/60 < 5) total
from sessions
union
select "[5-10>" bin, sum(duration/60 >= 5 and duration/60 < 10) total
from sessions
union
select "[10-15>" bin, sum(duration/60 >= 10 and duration/60 < 15) total
from sessions
union
select "15 or more" bin, sum(duration/60 >= 15) total
from sessions
```



## 1173. Immediate Food Delivery I (Easy)
Table: `Delivery`
```
+-----------------------------+---------+
| Column Name                 | Type    |
+-----------------------------+---------+
| delivery_id                 | int     |
| customer_id                 | int     |
| order_date                  | date    |
| customer_pref_delivery_date | date    |
+-----------------------------+---------+
```
delivery_id is the primary key of this table.
The table holds information about food delivery to customers that make orders at some date and specify a preferred delivery date (on the same order date or after it).


If the preferred delivery date of the customer is the same as the order date then the order is called immediate otherwise it's called scheduled.

Write an SQL query to find the percentage of immediate orders in the table, rounded to 2 decimal places.

The query result format is in the following example:

`Delivery` table:
```
+-------------+-------------+------------+-----------------------------+
| delivery_id | customer_id | order_date | customer_pref_delivery_date |
+-------------+-------------+------------+-----------------------------+
| 1           | 1           | 2019-08-01 | 2019-08-02                  |
| 2           | 5           | 2019-08-02 | 2019-08-02                  |
| 3           | 1           | 2019-08-11 | 2019-08-11                  |
| 4           | 3           | 2019-08-24 | 2019-08-26                  |
| 5           | 4           | 2019-08-21 | 2019-08-22                  |
| 6           | 2           | 2019-08-11 | 2019-08-13                  |
+-------------+-------------+------------+-----------------------------+
```
Result table:
```
+----------------------+
| immediate_percentage |
+----------------------+
| 33.33                |
+----------------------+
```
The orders with delivery id 2 and 3 are immediate while the others are scheduled.


```mysql
select round(100 * sum(order_date = customer_pref_delivery_date) / count(*), 2) immediate_percentage
from Delivery;
```


## 1098. Unpopular Books (Medium)
Table: `Books`
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| book_id        | int     |
| name           | varchar |
| available_from | date    |
+----------------+---------+
```
book_id is the primary key of this table.

Table: `Orders`
```
+----------------+---------+
| Column Name    | Type    |
+----------------+---------+
| order_id       | int     |
| book_id        | int     |
| quantity       | int     |
| dispatch_date  | date    |
+----------------+---------+
```
order_id is the primary key of this table.
book_id is a foreign key to the Books table.


Write an SQL query that reports the books that have sold less than 10 copies in the last year, excluding books that have been available for less than 1 month from today. Assume today is 2019-06-23.

The query result format is in the following example:

`Books` table:
```
+---------+--------------------+----------------+
| book_id | name               | available_from |
+---------+--------------------+----------------+
| 1       | "Kalila And Demna" | 2010-01-01     |
| 2       | "28 Letters"       | 2012-05-12     |
| 3       | "The Hobbit"       | 2019-06-10     |
| 4       | "13 Reasons Why"   | 2019-06-01     |
| 5       | "The Hunger Games" | 2008-09-21     |
+---------+--------------------+----------------+
```
`Orders` table:
```
+----------+---------+----------+---------------+
| order_id | book_id | quantity | dispatch_date |
+----------+---------+----------+---------------+
| 1        | 1       | 2        | 2018-07-26    |
| 2        | 1       | 1        | 2018-11-05    |
| 3        | 3       | 8        | 2019-06-11    |
| 4        | 4       | 6        | 2019-06-05    |
| 5        | 4       | 5        | 2019-06-20    |
| 6        | 5       | 9        | 2009-02-02    |
| 7        | 5       | 8        | 2010-04-13    |
+----------+---------+----------+---------------+
```

Result table:
```
+-----------+--------------------+
| book_id   | name               |
+-----------+--------------------+
| 1         | "Kalila And Demna" |
| 2         | "28 Letters"       |
| 5         | "The Hunger Games" |
+-----------+--------------------+
```


```mysql
select books.book_id, books.name
from books
left join (
    select book_id, sum(quantity) total_quantity
    from orders
    where dispatch_date > '2018-06-23'
    group by book_id) tmp
on tmp.book_id = books.book_id
where (total_quantity < 10 or total_quantity is null)
    and books.available_from < '2019-05-23'
order by 1;
```


